In [1]:
import os

In [2]:
%pwd

'e:\\MLProject\\chicken-disease\\research'

In [3]:
os.chdir("../")


In [4]:
%pwd

'e:\\MLProject\\chicken-disease'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class PrepareCallbacksConfig:
    root_dir: Path 
    tensorboard_root_log_dir: Path 
    checkpoint_model_filepath: Path 

In [12]:
from CNNClassifier.constants import *
from CNNClassifier.utils.common import create_directories,read_yaml



In [15]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_prepare_callback_config(self) ->PrepareCallbacksConfig:
        config=self.config.prepare_callback
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)

        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_call_back_config=PrepareCallbacksConfig(

            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_call_back_config

In [9]:
import os
import urllib.request
from zipfile import ZipFile
import tensorflow as tf
import time

In [ ]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config= config

    def _create_tb_callback(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir=os.path.join(self.config.tensorboard_root_log_dir,
                                        f"tb_logs_at{timestamp}")
        
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return[
            self._create_tb_callback,
            self._create_ckpt_callbacks
        ]

In [25]:
try:
    config=ConfigurationManager()
    prepare_callbacks_config=config.get_prepare_callback_config()
    prepare_callbacks=PrepareCallback(config=prepare_callbacks_config)
    callbacklist=prepare_callbacks.get_tb_ckpt_callbacks()

except Exception as e:
    raise e

[2026-01-26 18:05:04,215 - CNNClassifier - INFO - yaml file: config\config.yaml loaded successfully]
[2026-01-26 18:05:04,221 - CNNClassifier - INFO - yaml file: params.yaml loaded successfully]
[2026-01-26 18:05:04,224 - CNNClassifier - INFO - Directory created at: artifacts]
[2026-01-26 18:05:04,226 - CNNClassifier - INFO - Directory created at: artifacts\prepare_callback\checkpoints_dir]
[2026-01-26 18:05:04,228 - CNNClassifier - INFO - Directory created at: artifacts\prepare_callback\tensorboard_logs_dir]
